In [1]:
from mydataset import TRANSFORMER_DATA_MINDS,TRANSFORMER_ALL_DATA_MINDS
from mymodel import mymodel, Discriminator, backboneDiscriminator
from myloss import advLoss
from tqdm import tqdm
import random
import numpy
import os 
import mindspore.dataset as ds
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Model,ParameterTuple

import numpy as np

import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Model
from mindspore import dataset as ds
from mindspore.nn import LossBase
from mindspore.common.initializer import Normal
from mindspore.train.callback import LossMonitor


In [41]:
dis = Discriminator()
from mindspore import grad
class BabyModel(nn.Cell ):
    def __init__(self, auto_prefix=True, flags=None, in_features=24):
        super(BabyModel,self).__init__(auto_prefix, flags)
    
    def construct(self, x):
        return x
    

class BabyModel1(nn.Cell ):
    def __init__(self, auto_prefix=True, flags=None, in_features=24):
        super(BabyModel1,self).__init__(auto_prefix, flags)
    
    def construct(self, x):
        return x
    
    def bprop(x,out,dout):
        return (-dout,)

def getBackward(net,x):
    return grad(net)(x)



In [43]:
from mindspore import Tensor
from mindspore import dtype 
a = Tensor(np.random.randn(200,24),dtype=dtype.float32)
net = BabyModel()
net1 = BabyModel1()
getBackward(net,a)
print('11111')
net1.bprop_debug = True
getBackward(net1,a)

11111


TypeError: Size of bprop func inputs[0] is not equal to the size of cell inputs[1]

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/pipeline/pynative/grad/grad.cc:730 GetCustomBpropPrim


In [33]:
a 


Tensor(shape=[200, 24], dtype=Float32, value=
[[-1.45801306e+00, -1.67722797e+00,  2.93624461e-01 ...  3.75781238e-01,  1.10071361e-01,  2.64292836e-01],
 [-5.71087897e-01,  2.70826608e-01, -5.34282029e-01 ...  4.33681309e-01, -2.62964834e-02, -7.24124908e-01],
 [ 9.80217159e-01, -1.36771724e-01,  5.59742093e-01 ... -7.35678315e-01,  5.97096384e-01,  8.39291036e-01],
 ...
 [-3.45177382e-01,  3.43842119e-01,  2.39282772e-01 ...  2.66351163e-01, -1.61633599e+00, -8.47166181e-01],
 [-1.27451825e+00, -1.28423452e-01,  2.37298980e-01 ...  2.28835717e-01,  1.00056565e+00, -5.77156126e-01],
 [ 6.32132411e-01,  8.69370401e-01, -6.06274128e-01 ... -4.70208794e-01, -1.28252101e+00,  1.48819298e-01]])

In [5]:
a_res = dis(a)

In [6]:
a_res

Tensor(shape=[200, 1], dtype=Float32, value=
[[ 5.00809193e-01],
 [ 4.98660237e-01],
 [ 5.00307322e-01],
 ...
 [ 5.02206683e-01],
 [ 4.94636983e-01],
 [ 4.97292012e-01]])

In [8]:
dis_grad = ops.GradOperation(get_all = True)

In [9]:
dis_grader = dis_grad(dis)

In [25]:
dis_grader(a)

(Tensor(shape=[200, 24], dtype=Float32, value=
 [[ 8.22985021e-05,  4.18762684e-05, -4.58290888e-05 ... -4.37750743e-04,  5.82555076e-04,  4.08207736e-04],
  [ 9.33665433e-07,  2.09466831e-04, -1.23812235e-04 ... -5.94752491e-04,  8.44921451e-04,  1.86517893e-04],
  [-3.15542886e-04,  1.52259090e-04,  7.54803623e-05 ... -6.32163254e-04,  4.47979255e-05,  3.60304475e-05],
  ...
  [ 1.76842150e-05, -3.11468029e-04,  5.20976901e-05 ... -4.68352519e-04,  3.80118465e-04,  1.86900172e-04],
  [ 1.94627006e-04, -2.81962057e-05,  5.79072439e-05 ... -6.67542219e-04,  7.91916798e-04, -1.77568829e-04],
  [ 5.93697841e-05,  3.45557521e-04,  6.32152369e-05 ... -4.00470395e-04,  2.56589236e-04,  5.17265653e-06]]),)

In [19]:
from mindspore import Parameter
from mindspore.common import ParameterTuple
from mindspore.ops import GradOperation
from mindspore import dtype as mstype
from mindspore.ops import operations as P
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.matmul = P.MatMul()
        self.z = Parameter(Tensor(np.array([1.0], np.float32)), name='z')
    def construct(self, x, y):
        # x = x * self.z
        out = self.matmul(x, y)
        return out

class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.grad_op = GradOperation()
    def construct(self, x, y):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x, y)

x = Tensor([[0.5, 0.6, 0.4], [1.2, 1.3, 1.1]], dtype=mstype.float32)
y = Tensor([[0.01, 0.3, 1.1], [0.1, 0.2, 1.3], [2.1, 1.2, 3.3]], dtype=mstype.float32)
output = GradNetWrtX(Net())(x, y)

In [20]:
x

Tensor(shape=[2, 3], dtype=Float32, value=
[[ 5.00000000e-01,  6.00000024e-01,  4.00000006e-01],
 [ 1.20000005e+00,  1.29999995e+00,  1.10000002e+00]])

In [21]:
y

Tensor(shape=[3, 3], dtype=Float32, value=
[[ 9.99999978e-03,  3.00000012e-01,  1.10000002e+00],
 [ 1.00000001e-01,  2.00000003e-01,  1.29999995e+00],
 [ 2.09999990e+00,  1.20000005e+00,  3.29999995e+00]])

In [22]:
output

Tensor(shape=[2, 3], dtype=Float32, value=
[[ 1.41000009e+00,  1.59999990e+00,  6.59999990e+00],
 [ 1.41000009e+00,  1.59999990e+00,  6.59999990e+00]])